# 내일배움캠프 46일차 TIL + Stramlit, Python

## 목차

오늘의 TIL 순서는

1. **시작전 마음가짐**
2. **Streamlit 구현**
3. **수준별 학습반**


### 코드는 GITHUB 링크를 확인해주세요
- streamlit 구현: https://github.com/maricecil/RAG_project/blob/1-%EC%9E%A5%ED%95%9C%EA%B2%80/app.py

- Python 문법: https://github.com/Onesword-Jang/TIL/tree/main/basic
---------------

## 시작전 마음가짐
어제 TIL을 보니 너무 TIL에 소홀해진 느낌이 들었습니다.

진행되는 과정 중 새롭게 알게 된 내용이나 트러블이 나온다면 제대로 기록하는 것이 목표입니다.

또한 스트림릿을 받은 코드로 구현하는 것이 최종 목표입니다.

학습을 시작하겟습니다.

---------------

## Streamlit 구현

팀원의 코드를 받아서 카테고리를 정하고 질문지를 가져와 사용자가 선택해서 채팅형태로 대답을 하고 튜터링을 받는 웹사이트를 구현하는 것을 목표로 진행하고있습니다.

오늘 시작은 준기님의 코드를 파악하고 사용자 지정함수를 만드는 과정, 지정 함수를 어떤 기능에 어떻게 집어넣을지 파악하였습니다.

```python
# PDF 처리 함수
def process_pdf(file_path, api_key):
    """PDF 파일을 처리하여 텍스트 청크를 반환."""
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    split_texts = text_splitter.split_documents(documents)
    return split_texts

# 질문 파일 로드 함수
def load_questions(topic):
    """카테고리별 질문 파일을 로드."""
    file_name = topic_file_mapping.get(topic)
    if not file_name:
        return []
    questions_file = os.path.join(questions_folder, file_name)
    if not os.path.exists(questions_file):
        return []
    with open(questions_file, "r", encoding="utf-8") as file:
        questions = file.readlines()
    return [q.strip() for q in questions if q.strip()]  # 공백 제거

# FAISS 저장 및 로드 함수
def save_faiss_index(vectorstore, topic):
    """FAISS 인덱스를 주제별로 저장."""
    sanitized_topic = topic_file_mapping.get(topic, topic).replace("_module.txt", "")
    index_path = os.path.join(faiss_index_folder, f"{sanitized_topic}_index")
    os.makedirs(index_path, exist_ok=True)
    vectorstore.save_local(index_path)

def load_faiss_index(topic, api_key):
    """주제별로 저장된 FAISS 인덱스를 로드."""
    sanitized_topic = topic_file_mapping.get(topic, topic).replace("_module.txt", "")
    index_path = os.path.join(faiss_index_folder, f"{sanitized_topic}_index")
    if os.path.exists(index_path):
        embeddings = OpenAIEmbeddings(openai_api_key=api_key)
        return FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
    return None
```

4가지 사용자 지정 함수를 만들었습니다.

이를 활용할 수 있게 파일 경로와 카테고리 이름을 file_path로 매핑해서 저장 하였고 질문지 또한 마찬가지로 진행하였습니다.

현재 기초적으로 작동하는 부분은 구현을 마쳤습니다.

그리고 지금까지 해온게 킴원분의 코드에서 많이 벗어난다는 느낌이 들어 팀원분의 의견을 생각하며 다른게 무엇인지 고민해보았습니다.

1. faiss에 문서들이 저장 된 형태라서 1번 문서를 불러오면 다음부터는 필요가 없음.
2. 다국어 지원이 가능 함
3. 대화 내용을 기억하고 관리 할 수 있음

지금 내가 구현 하지 못한 부분이기도 하면서 많이 다르다고 느껴진게 1번입니다.

그래서 배포를 할 때는 최초의 한번 데이터를 불러오고 저장하는 방법이 없기 때문에 해결 방법을 고민해 보았습니다.

- solution
 - 데이터베이스 활용
 - 클라우드 스토리지 활용
 - REST API로 데이터 제공
 - 벡터 데이터 저장 및 검색
 
이렇게 검색의 되었고 가장 단순한 방법이 **벡터 데이터 저장 및 검색**이라고 생각을 했고 이는 문서에서 text테이터를 추출해 그 데이터를 리스트로 저장하여 같이 배포하는 형태였습니다.

우선 이렇게 내 마음대로 해도 될지가 고민이였지만 지금 고민할거리가 아니라는 생각에 이제는 프롬프트 적용하는 방법을 구현해야 할듯 합니다.

------------------

## Python 문법 수업
오늘은 파이썬 기초 문법을 공부하는 시간을 가졌습니다.

튜터님께서 만들어주신 문제를 풀어보다가 여러가지 인자받기에서 어려움을 느꼇습니다.

--------------

## 회고 

시간이 늦어 회고는 조금 짧습니다.

어떤걸 구현할지 어떻게 구현할지 감이 잡히지 않아 일단 필수 구현부분을 GPT의 도움을 받아 작성하고있었습니다.

제가 맏은 세션의 챗봇의 기동은 성공적이였지만 아직 질문 선택시 대답하기 전에 답을 말해버리는 오류가 생겼습니다.

분명 잘되었는데 이를 수정할 방법을 생각하다가 같은 팀원분이 이대로가면 완성을 못할 것 같다고 한가지 세션을 같이 고민하자는 의견을 듣고 같이 화면 공유를 하며 오류를 해결해보았고 이는 괸장히 성공적이였습니다.

그래서 우선 같이 한 세션을 구현하기로 했습니다.

감사합니다.
